# Исследование надежности заемщиков

**Задача:**

Необходимо выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

**Данные:**

Статистика о платёжеспособности клиентов.

## Данные и общая информация

Изучим данные предоставленные для исследования.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


*Описание данных*

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Предобработка данных

### Обработка аномальных значений

В данных встречаются артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом является отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце и заменим все отрицательные значения положительными с помощью метода `abs()`.

In [5]:
data['days_employed'] = data['days_employed'].abs()

Проверим так же уникальные значения в столбце `children`.

In [6]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Здесь присутствует два аномальных значения. Удалим строки с ними.

In [7]:
data = data[(data['children'] != 20) & (data['children'] != -1)]

Проверим уникальные значения.

In [8]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков

Проверим данные на наличие пропусков.

In [9]:
data.isna().sum()

children               0
days_employed       2162
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2162
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения.

Один из них — `days_employed`. Пропуски в этом столбце заполним медианными значениями по каждому типу занятости из столбца `income_type`.

Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце так же необходимо медианным значением по каждому типу из столбца `income_type`. 

In [10]:
for i in data['income_type'].unique():
    data.loc[(data['income_type'] == i) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == i), 'days_employed'].median()


In [11]:
for i in data['income_type'].unique():
    data.loc[(data['income_type'] == i) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == i), 'total_income'].median()

Проверим наличие пропусков после операции над ними.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [13]:
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем все значения в столбце к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()

Проверим данные на наличие строк-дубликатов и если таковые имеются, удалим их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме столбец `total_income_category` с категориями для разделения клиентов по категориям доходов:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.


In [17]:
def categorize_income(money):
    if money <= 30000:
        return 'E'
    elif 30001 <= money <= 50000:
        return 'D'
    elif 50001 <= money <= 200000:
        return 'C'
    elif 200001 <= money <=1000000:
        return 'B'
    else:
        return 'A'

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Сформируем навый столбец `purpose_category` на основании данных из столбца `purpose`, в которые войдут следующие категории, в зависимости от целей взятия кредита.

- "операции с автомобилем"
- "операции с недвижимостью"
- "проведение свадьбы"
- "получение образования"

Для этого сначала взглянем на перечень уникальных целей взятия кредита и создадим функцию для формирования нового столбца.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(category):
    if 'автомобил' in category:
        return 'операции с автомобилем'
    elif 'свадьб' in category:
        return 'проведение свадьбы'
    elif 'жиль' in category or 'недвижимост' in category:
        return 'операции с недвижимостью'
    elif 'образован' in category:
        return 'получение образования'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

**Вывод:**
Данные подготовлены, готовы для дальнейшего исследования и ответов на вопросы.

## Исследование данных и ответы на вопросы

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data.groupby('children')['debt'].mean()

children
0    0.075438
1    0.092346
2    0.094542
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

**Вывод:** 
Зависимость между количеством детей и возвратом кредита в срок есть. Чем меньше детей, тем меньше просрочек по кредиту. Но смотря на сгруппированные данные может показаться, что это не так, так как у семей с 5 детьми 0 просрочек по кредиту. Дело в том, что в данных количество семей с 5 детьми очень мало и будет неправильно сравнивать зависимость с таким значением данных.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data.groupby('family_status')['debt'].mean()

family_status
Не женат / не замужем    0.097639
в разводе                0.070648
вдовец / вдова           0.066246
гражданский брак         0.093130
женат / замужем          0.075606
Name: debt, dtype: float64

**Вывод:** 
Зависимость между семейным положением и возвратом кредита в срок имеется. Одинокие люди имеют больше просрочек (Не женат / не замужем - 0.097), чем семейные пары, наименьшие просрочки по кредитам имеют вдовец / вдова - 0.06.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
data.groupby('total_income_category')['debt'].mean()

total_income_category
A    0.080000
B    0.070602
C    0.084982
D    0.060172
E    0.090909
Name: debt, dtype: float64

**Вывод:** 
Зависимости между уровнем дохода и возвратом кредита в срок как таковой и нет. Больше всего просрочек имеет группа с уровнем дохода Е, что в принципе может звучать очевидно, но за ними идет группа дохода А и С. Меньше всего просрочек у группы дохода D, что при ожидаемой зависимости могли бы идти после группы E.

### Как разные цели кредита влияют на его возврат в срок?

In [25]:
data.groupby('purpose_category')['debt'].mean()

purpose_category
операции с автомобилем      0.093480
операции с недвижимостью    0.072551
получение образования       0.092528
проведение свадьбы          0.079118
Name: debt, dtype: float64

**Вывод:** 
Самыми надежными кредитом являются кредит на операции с недвижимость(0.072). Самым не надежным - кредит на операции с автомобилем.

### Причины появления пропусков в исходных данных.

В исходных данных присутствуют пропуски только в двух столбцах: `days_employed` и `total_income`. Эти строки хранят в себе информацию о трудовом стаже и данные о доходах. А так же при том, что их одинаковое кооличество (2174), можно предположить, что этих данных просто нет, т.к. эти люди не работали. Так же, в этих столбцах нет значений равных нулю, так что можно предположить, что пропуски это нули.

### Почему заполнять пропуски медианным значением - лучшее решение для количественных переменных.

В данных с количественными переменными могут быть самые разные числа, будь то почти все близкие друг к другу росты людей, либо зарплаты по региону, где у одних 15000 рублей, а у пары человек 1000000 рублей, и эти пару человек перетягивают на себя одеяло, называемым средним значение выборки. А при медианным значением найдется ровно та половина выборки, которая расположенна по порядку, которая необходима.

## Общий вывод

Было проведено исследование, по выявлению влияния разных факторов на факт погашения кредита в срок. Результаты исследования помогут при построении модели кредитного скоринга - специальной системы, которая оценивает способность потенциального заемщика вернуть кредит банку. Входные данные - статистика о платежеспособности клиентов.

Перед проведением исследования были поставлены 4 гипотезы:
- Гипотеза 1: как количество детей влияет на факт погашения кредита;
- Гипотеза 2: как семейное положение влияет на факт погашения кредита;
- Гипотеза 3: как уровень дохода влияет на факт погашения кредита;
- Гипотеза 4: как цель кредита влияет на факт погашения кредита.

**Гипотеза 1 частично потдверждена:** *для более точных выводов необходимо больше данных о семьях с 3, 4 и 5 детьми.
- у заемщиков без детей 7.5% доля просроченных кредитов;
- у заемщиков 1 ребенком 9.2% доля просроченных кредитов;
- у заемщиков 2 детьми 9.5% доля просроченных кредитов.

**Гипотеза 2 подтверждена:**
- у не женатых заемщиков (9.8%) и тех, у кого гражданский брак (9.3%) выше вероятность быть должниками.
- те, кто в разводе (7.6%) или овдовел (6.6%) меньшая вероятность стать должником.

**Гипотеза 3 частично подтверждена:** *для более точных выводов необходимо больше данных о клиентах с уровнями дохода катигорий D(от 30001 до 50000), A(от 1000001) и E(до 30000)
- у заемщиков с уровнем дохода категории B(от 200001 до 1000000) доля просроченных кредитов составляет 7.1%;
- у заемщиков с уровнем дохода категории С(от 500001 до 200000) доля просроченных кредитов составляет 8.5%.

**Гипотеза 4 потдверждена:**
- кредиты для операции с недвижимостью. Доля просроченных составляет 7.3%;
- кредиты для проведения свадьбы. Доля просроченных составляет 7.9%;
- кредиты для операции с автомобилем. Доля просроченных составляет 9.3%;
- кредиты для получения образования. Доля просроченных составляет 9.3%;

**Рекомендации перед составлением кредитного скоринга:**
1. За основу в построении модели скоринга можно использовать зависимости на основе второй и четвертой гипотезы (семейное положение и цель кредита);
2. Для проверки по первой и третьей гипотезы необходимо собрать больше данных.